In [1]:
import pandas as pd 
import numpy as np 
import scipy.sparse as sparse
from scipy.sparse.linalg import spsolve

### Loading our data 

In [2]:
retail_data = pd.read_excel("Online Retail.xlsx")

In [3]:
retail_data.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom


In [4]:
retail_data['StockCode'].count()

541909

In [5]:
retail_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 541909 entries, 0 to 541908
Data columns (total 8 columns):
InvoiceNo      541909 non-null object
StockCode      541909 non-null object
Description    540455 non-null object
Quantity       541909 non-null int64
InvoiceDate    541909 non-null datetime64[ns]
UnitPrice      541909 non-null float64
CustomerID     406829 non-null float64
Country        541909 non-null object
dtypes: datetime64[ns](1), float64(2), int64(1), object(4)
memory usage: 33.1+ MB


##### from the above we can drop the rows that have NaN customerId, then check our data again 

In [7]:
cleaned_data = retail_data[retail_data['CustomerID'].notnull()]

In [8]:
cleaned_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 406829 entries, 0 to 541908
Data columns (total 8 columns):
InvoiceNo      406829 non-null object
StockCode      406829 non-null object
Description    406829 non-null object
Quantity       406829 non-null int64
InvoiceDate    406829 non-null datetime64[ns]
UnitPrice      406829 non-null float64
CustomerID     406829 non-null float64
Country        406829 non-null object
dtypes: datetime64[ns](1), float64(2), int64(1), object(4)
memory usage: 27.9+ MB


Before we make any sort of ratings matrix, it would be nice to have a lookup table that keeps track of each item ID along with a description of that item. Let’s make that now.

In [11]:
item_lookup = cleaned_data[['StockCode','Description']].drop_duplicates()
item_lookup['StockCode'] = item_lookup.StockCode.astype(str)  # this encodes stockcode as string for easy lookup 

In [12]:
type(item_lookup['StockCode'])

pandas.core.series.Series

In [16]:
cleaned_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 406829 entries, 0 to 541908
Data columns (total 8 columns):
InvoiceNo      406829 non-null object
StockCode      406829 non-null object
Description    406829 non-null object
Quantity       406829 non-null int64
InvoiceDate    406829 non-null datetime64[ns]
UnitPrice      406829 non-null float64
CustomerID     406829 non-null float64
Country        406829 non-null object
dtypes: datetime64[ns](1), float64(2), int64(1), object(4)
memory usage: 27.9+ MB


#### We prepare our data for sparse matrix 

In [19]:
cleaned_data['CustomerID'] = cleaned_data.CustomerID.astype(int)


C:\Users\Mac\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [21]:
cleaned_data['CustomerID'].dtype

dtype('int32')

In [23]:
cleaned_retail = cleaned_data[['CustomerID','StockCode','Quantity']]
cleaned_retail.head()

,CustomerID,StockCode,Quantity
0,17850,85123A,6
1,17850,71053,6
2,17850,84406B,8
3,17850,84029G,6
4,17850,84029E,6


In [29]:
group_cleaned = cleaned_retail.groupby(['CustomerID','StockCode']).sum()
group_cleaned.head()

Quantity
CustomerID StockCode          
12346      23166             0
12347      16008            24
           17021            36
           20665             6
           20719            40

In [30]:
group_cleaned['Quantity'].loc[group_cleaned['Quantity'] == 0 ] = 1 #replace the Zeros with ones for purchases 

In [31]:
group_cleaned['Quantity'].head()

CustomerID  StockCode
12346       23166         1
12347       16008        24
            17021        36
            20665         6
            20719        40
Name: Quantity, dtype: int64

In [34]:
grouped_purchased = group_cleaned.query('Quantity > 0') # Only get customers where purchase totals were positive
grouped_purchased.head()

Quantity
CustomerID StockCode          
12346      23166             1
12347      16008            24
           17021            36
           20665             6
           20719            40